In [ ]:
#Import all the basic packages
import pandas as pd
import spotipy
import matplotlib
from spotipy.oauth2 import SpotifyOAuth
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
#Spotipy API user authentication
cid = 'f6ade21d86f6443891372a007434a75f'
secret = 'df4a7b00a1a64cb7b4b5c4dfa538b095'
redirect_uri = 'https://localhost:3000/'
scope = 'user-top-read playlist-modify-private playlist-modify-public'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cid,
                                               client_secret=secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

In [ ]:
#grepping the username
username = sp.current_user()['href'].split('/')[5]

In [ ]:
#take in raw inputs for link and name
playlist_name = input("Playlist Name: ")
playlist_link = input("Playlist Link: ")
print("Creating {} from {}".format(playlist_name,playlist_link))

In [ ]:
#split it just to get the uri
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
#get the first uri
uri = sp.playlist_tracks(playlist_URI)["items"][0]["track"]["uri"]
#create dataframe and input the first
df = pd.DataFrame(sp.audio_features(uri))
#loop through and add the rest with the artist info
track_name = []
artist_name = []
artist_pop = []
track_pop = []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    uri = track["track"]["uri"]
    spi = sp.audio_features(uri)
    df = df.append(other=pd.DataFrame(spi), ignore_index=True)
    #artist info grep
    track_name.append(track["track"]["name"])
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    artist_name.append(track["track"]["artists"][0]["name"])
    artist_pop.append(artist_info["popularity"])
    track_pop.append(track["track"]["popularity"])
#take the first row off and reset the index
df = df.drop_duplicates(ignore_index=True)
#add the other columns
df['track_name'] = track_name
df['artist_name'] = artist_name
df['artist_pop'] = artist_pop
df['track_pop'] = track_pop
#remove the unnecessary data
drop_cols = ['mode', 'time_signature', 'analysis_url', 'uri', 'track_href', 'type', 'duration_ms']
df = df.drop(drop_cols, axis=1)
#reorder the columns
df = df[['track_name', 'artist_name', 'artist_pop', 'track_pop', 'id', 'danceability', 'energy', 'key', 'loudness',
         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [ ]:
df.head()

In [ ]:
#using the Spotipy rec api to get more data on songs similiar to the ones in the track (this can be skipped)
rec_song = []
for i in df['id'].values.tolist():
    rec_song += sp.recommendations(seed_tracks=[i], limit=5)['tracks']
rec_song_id = []
rec_song_name = []
rec_artist_name = []
rec_artist_pop = []
rec_song_pop = []
for i in rec_song:
    rec_song_id.append(i['id'])
    rec_song_name.append(i['name'])
    rec_artist_name.append(i["artists"][0]["name"])
    artist_uri = i["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    rec_artist_pop.append(artist_info["popularity"])
    rec_song_pop.append(i["popularity"])

rec_song_features = []
for i in range(0, len(rec_song_id)):
    rec_audio_features = sp.audio_features(rec_song_id[i])
    for track in rec_audio_features:
        rec_song_features.append(track)
rec_song_playlist = pd.DataFrame(rec_song_features)
rec_song_playlist['track_name'] = rec_song_name
rec_song_playlist['artist_name'] = rec_artist_name
rec_song_playlist['artist_pop'] = rec_artist_pop
rec_song_playlist['track_pop'] = rec_song_pop
rec_song_playlist = rec_song_playlist.drop(drop_cols, axis=1)
rec_song_playlist = rec_song_playlist[
    ['track_name', 'artist_name', 'artist_pop', 'track_pop', 'id', 'danceability', 'energy', 'key', 'loudness',
     'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [ ]:
rec_song_playlist.head()

In [ ]:
#remove duplicates
df.drop_duplicates(subset=['id'])
#add the favorite column and add 1 to all of the values
df['favorite'] = 1

In [ ]:
#refine the rec song df
rec_song_refined = rec_song_playlist.drop(columns=['track_name', 'artist_name', 'artist_pop', 'track_pop'])
#read csv data for all spotify data
spotify_df_pre = pd.read_csv('spotify_songs.csv')
#combine the two for the entire dataframe
frames_total = [rec_song_refined,spotify_df_pre]
spotify_df = pd.concat(frames_total)
#add the favorite column with zero
spotify_df['favorite'] = 0

In [ ]:
#make a new data frame with just the data values (the other fave_df id just to see
refined_fave_df = df.drop(columns=['track_name', 'artist_name', 'artist_pop', 'track_pop'])

In [ ]:
#combine them
total_frames = [spotify_df, refined_fave_df]
combined_df = pd.concat(total_frames)
combined_df = combined_df.dropna(how='any', axis=0)

In [ ]:
#finally we will create the model
#shuffle the dataset
shuffled_c_df = combined_df.sample(frac=1)
#make a size for the split
split_size = int(0.8 * len(combined_df))
#create the splits
training_df = shuffled_c_df[:split_size]
testing_df = shuffled_c_df[split_size:]

In [ ]:
#set the X and y
X = training_df.drop(columns=['id', 'favorite'])
y = training_df['favorite']
#refine the test set
X_test = testing_df.drop(columns=['id', 'favorite'])
y_test = testing_df['favorite']

In [ ]:
#resample using the Synthetic Minority Oversampling Technique to get more favorite songs since it is a ratio of 500 to 8000
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X, y)

In [ ]:
#train to plots the confusion matrix (hyperparameters found in previous tests --> now are set)
dt = DecisionTreeClassifier(max_depth=30).fit(X_train, y_train)
#plot a confusion matrix
plot_confusion_matrix(dt, X_train, y_train)

In [ ]:
#created pipeline to scale te data and re
pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=30))
pipe.fit(X_train, y_train)
Pipeline(steps=[('Scaler', StandardScaler()),('Tree', DecisionTreeClassifier(max_depth=30))])
pipe.score(X_test, y_test)

In [ ]:
#Created a threshold for user to change to get more or less songs
predict_proba = pipe.predict_proba(combined_df.drop(['favorite', 'id'], axis=1))
accuracy = 0.999  # threshold can be changed depending on user
length_of_predict_proba = len(predict_proba) #length
preds = [1 if predict_proba[i][1] > accuracy else 0 for i in range(length_of_predict_proba)]
combined_df['prediction'] = preds

In [ ]:
#create the datafram with 0 favorite (in database not OG playlist ) and prediction of 1 (songs predicted to fit the model)
playlist_df = combined_df[(combined_df['favorite'] == 0) & (combined_df['prediction'] == 1)]

In [ ]:
#make them a list to pass in a future function
playlist_track_list = playlist_df['id'].tolist()

In [ ]:
#create the playlist in the users spotify
playlist = sp.user_playlist_create(user=username, name=playlist_name)
#get the ID for the playlist to add songs
playlist_id = playlist['id']

In [ ]:
#add the songs
playlist_final = sp.playlist_add_items(playlist_id, playlist_track_list)

In [ ]:
#get the url for the playlist
playlist_url = (sp.current_user_playlists()['items'][0]['external_urls']).values()

In [ ]:
url = "https://open.spotify.com/embed/playlist/"+str(playlist_url)[48:].removesuffix("'])")+"?utm_source=generator"

In [ ]:
final_url = "<iframe src={} width={} height={} frameBorder={}></iframe>".format(url,"100%","380","0")

In [ ]:
from IPython.display import HTML

HTML(final_url)